In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import bs4

In [3]:
# Define Slug Extraction Function

def extract_slugs (max_page_id):

  # Initialize empty list of coffee review names and slugs
  name_slug_list = []


  # Iterate through 257 pages of coffee reviews
  print('Begin Scraping!')
  for i in range(1,max_page_id+1):
      url = f'https://www.coffeereview.com/advanced-search/page/{i}/'
      res = requests.get(url,headers = {'User-agent': 'Lisa Vanderpump'})
      soup = BeautifulSoup(res.content, "lxml")
      names = soup.find_all('p',{'class':"review-title"})
      
      #collect name and slugs
      for name in names:
          ns_dict ={}
          ns_dict['name'] = name.text
          ns_dict['slug'] = name.find('a').attrs['href']
          
          #add dictionary to list
          name_slug_list.append(ns_dict)
      
      #counter
      if (i%25) == 0:
          print(f'scraped {i} pages')
          #pause between requests
          time.sleep(1)

  return name_slug_list

In [4]:
# Execute Function
name_slug_list = extract_slugs(115)

Begin Scraping!
scraped 25 pages
scraped 50 pages
scraped 75 pages
scraped 100 pages


In [5]:
# Export Slugs to Local Filesystem
name_slug_df=pd.DataFrame(name_slug_list)
name_slug_df.to_csv('name_slug.csv')

In [6]:
# Define Prcessing Function

def process_slug(url):
  results={}
  try:
    #url = 'https://www.coffeereview.com{}'.format(slug)
    res = requests.get(url,headers = {'User-agent': 'Kyle Richards'})
    soup = BeautifulSoup(res.content, "lxml")

    try:
      results['score'] = soup.find('span',{'class':'review-template-rating'}).text
    except:
      results['score']= 'Not Available'

    try:
      results['roaster'] = soup.find('p',{'class':'review-roaster'}).text
    except:
      results['roaster']= 'Not Available'

    try:
      results['title'] = soup.find('h1',{'class':'review-title'}).text
    except:
      results['title']= 'Not Available'   

    table_elements= soup.find_all('table',{'class':'review-template-table'})
    table_elements = pd.read_html(str(table_elements))[0]
    
    for i,j in enumerate(table_elements.iloc[:,0]):
      results[j.replace(':','')]=table_elements.iloc[:,1][i]

    for strong_tag in soup.find_all('strong'):
        if 'Blind Assessment' in strong_tag.text:
          results['blind_assessment']= strong_tag.next_sibling
        if 'Notes' in strong_tag.text:
          results['notes']= strong_tag.next_sibling
        if 'The Bottom Line' in strong_tag.text:
          results['bottom_line']= strong_tag.next_sibling
    return results
  except Exception as e:
    return e

In [7]:
review_data= []
problem_links = []
erros = []
for url in name_slug_df['slug']:
    try:
        df = pd.DataFrame(process_slug(url=url),index=[0])
        review_data.append(df)
        print(url)
    except Exception as e:
        erros.append(e)
        print("problema : ", url)
        problem_links.append(url)

https://www.coffeereview.com/review/gesha-spirits/
https://www.coffeereview.com/review/kenya-nyeri-hill-farm-aa-top/
https://www.coffeereview.com/review/philippines-sitio-belis-1017/
https://www.coffeereview.com/review/philippines-sitio-belis-1014/
https://www.coffeereview.com/review/costa-rica-tarrazu-dota-el-diosa-geisha/
https://www.coffeereview.com/review/ethiopia-gedeb-worka-single-variety-74110-g1/
https://www.coffeereview.com/review/costa-rica-las-montanas-honey/
https://www.coffeereview.com/review/colombia-finca-la-guaca-aji/
https://www.coffeereview.com/review/44%cb%9an-68%cb%9aw-bar-harbor/
https://www.coffeereview.com/review/panama-finca-santa-teresa/
https://www.coffeereview.com/review/zambia-mafinga-hills-natural/
https://www.coffeereview.com/review/mexico-altura-natural-water-process-decaf/
https://www.coffeereview.com/review/area-code-207/
https://www.coffeereview.com/review/kenya-washed-kiambu-uklili-ab-espresso/
https://www.coffeereview.com/review/karatina-kenya-2/
htt

In [8]:
len(problem_links)

243

In [9]:
len(review_data)

1921

In [11]:
reviews = pd.concat(review_data)
reviews.to_csv("coffe_reviews_v2.csv")